In [1]:
!pip install optimum[exporters,onnxruntime] transformers onnxruntime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 4.2 MB/s eta 0:00:00


In [2]:
cd /content/drive/MyDrive/Local-Video-RAG

/content/drive/MyDrive/Local-Video-RAG


In [ ]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer, AutoProcessor
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
import shutil

model_id = "openai/clip-vit-base-patch32"
base_dir = "onnx_clip_model"
float32_dir = os.path.join(base_dir, "float32")
quantized_dir = os.path.join(base_dir, "quantized")


os.makedirs(float32_dir, exist_ok=True)
os.makedirs(quantized_dir, exist_ok=True)

print(f"--- 1. Exporting Float32 Model (Safe Mode) ---")

model = ORTModelForFeatureExtraction.from_pretrained(model_id, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)


model.save_pretrained(float32_dir)
tokenizer.save_pretrained(quantized_dir)
processor.save_pretrained(quantized_dir)
print("Float32 Export complete.")

print("--- 2. Manual Dynamic Quantization ---")



input_model_path = os.path.join(float32_dir, "model.onnx")

output_model_path = os.path.join(quantized_dir, "model_quantized.onnx")

quantize_dynamic(
    model_input=input_model_path,
    model_output=output_model_path,
    weight_type=QuantType.QUInt8
)

print(f"Quantization complete!")
print(f"Original Size: {os.path.getsize(input_model_path) / (1024*1024):.2f} MB")
print(f"Quantized Size: {os.path.getsize(output_model_path) / (1024*1024):.2f} MB")

Multiple distributions found for package optimum. Picked distribution: optimum
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


--- 1. Exporting Float32 Model (Safe Mode) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/transformers/models/clip/modeling_clip.py:197: Tra

Float32 Export complete.
--- 2. Manual Dynamic Quantization ---


Quantization complete!
Original Size: 577.48 MB
Quantized Size: 145.61 MB


In [ ]:
import shutil
from google.colab import files


shutil.make_archive('my_edge_ai_model', 'zip', 'onnx_clip_model')

files.download('my_edge_ai_model.zip')

In [4]:
import onnxruntime as ort
from transformers import AutoProcessor
from PIL import Image
import requests
import numpy as np
import os


quantized_model_path = "onnx_clip_model/quantized/model_quantized.onnx"
config_path = "onnx_clip_model/quantized"

print("--- 1. Loading ONNX Model via Raw ONNX Runtime ---")
try:

    session = ort.InferenceSession(quantized_model_path)
    print("✅ Model loaded successfully into ONNX Runtime!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit()

print("--- 2. Pre-processing Inputs ---")
processor = AutoProcessor.from_pretrained(config_path)


url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
texts = ["a photo of a cat", "a photo of a dog", "a car"]


inputs = processor(text=texts, images=image, return_tensors="np", padding=True)


ort_inputs = {
    "input_ids": inputs["input_ids"].astype(np.int64),
    "pixel_values": inputs["pixel_values"].astype(np.float32),
    "attention_mask": inputs["attention_mask"].astype(np.int64)
}

print("--- 3. Running Inference ---")

outputs = session.run(None, ort_inputs)


logits_per_image = outputs[0]
probs = (np.exp(logits_per_image) / np.sum(np.exp(logits_per_image), axis=1, keepdims=True))

print("\n--- ✅ Final Results ---")
for i, text in enumerate(texts):
    print(f"Confidence for '{text}': {probs[0][i]*100:.2f}%")


print("\n--- 🔍 Model Signature (Save this for C++) ---")
print("Inputs:")
for i in session.get_inputs():
    print(f"  - Name: {i.name}, Shape: {i.shape}, Type: {i.type}")

print("Outputs:")
for i in session.get_outputs():
    print(f"  - Name: {i.name}, Shape: {i.shape}, Type: {i.type}")

--- 1. Loading ONNX Model via Raw ONNX Runtime ---
✅ Model loaded successfully into ONNX Runtime!
--- 2. Pre-processing Inputs ---
--- 3. Running Inference ---

--- ✅ Final Results ---
Confidence for 'a photo of a cat': 98.42%
Confidence for 'a photo of a dog': 1.19%
Confidence for 'a car': 0.39%

--- 🔍 Model Signature (Save this for C++) ---
Inputs:
  - Name: input_ids, Shape: ['text_batch_size', 'sequence_length'], Type: tensor(int64)
  - Name: pixel_values, Shape: ['batch_size', 'num_channels', 'height', 'width'], Type: tensor(float)
  - Name: attention_mask, Shape: ['text_batch_size', 'sequence_length'], Type: tensor(int64)
Outputs:
  - Name: logits_per_image, Shape: ['image_batch_size', 'text_batch_size'], Type: tensor(float)
  - Name: logits_per_text, Shape: ['text_batch_size', 'image_batch_size'], Type: tensor(float)
  - Name: text_embeds, Shape: ['text_batch_size', 512], Type: tensor(float)
  - Name: image_embeds, Shape: ['image_batch_size', 512], Type: tensor(float)
